In [1]:
import csv
import os
import matplotlib.pyplot as plt
import re
import random
import numpy as np
import sqlite3 as db
import pandas as pd

######part1 从数据库中读取数据

def readFronSqllite(db_path,exectCmd):
    conn = db.connect(db_path)  # 该 API 打开一个到 SQLite 数据库文件 database 的链接，如果数据库成功打开，则返回一个连接对象
    cursor=conn.cursor()        # 该例程创建一个 cursor，将在 Python 数据库编程中用到。
    conn.row_factory=db.Row     # 可访问列信息
    cursor.execute(exectCmd)    #该例程执行一个 SQL 语句
    rows=cursor.fetchall()      #该例程获取查询结果集中所有（剩余）的行，返回一个列表。当没有可用的行时，则返回一个空的列表。
    return rows

def readfromAppaFrame(ARPAFrame):
    subARPA=ARPAFrame.split(',')
    return subARPA

def get_dataset(path='./db.sqlite3'):
    dataset = []
    for provinceID in [i + 1 for i in range(34)]:
        for year in [i + 2017 for i in range(3)]:
            for categoryID in range(1,4):
                sql = 'select score,rank from Rankings where provinceID_id=%s and year=%s and categoryID_id=%s' % (provinceID, year,categoryID)
                rows = readFronSqllite(path, sql)
                data = []
                data.append((provinceID, year,categoryID))
                for row in rows:
                    data.append(list(row))
                for i in range(len(data) - 2):
                    data[-(1 + i)][-1] = data[-(1 + i)][-1] - data[-(2 + i)][-1]
                dataset.append(data)
    return dataset

###part2 聚类算法以及其优化

def distance(x, y): #求距离
    return abs(x-y)

def Clustering(data, k):
    #data:分数列表[[分数1， 人数1],...,[分数n， 人数n]] ， k：超参量，簇的数目
    cluster = [] #创建簇中心集合的数组
    belong = []  #记录每一个分数隶属于哪一个中心
    for _ in range(k): #按照k值在cluster中创建代表簇中心数
        cluster.append(data[random.randint(0, len(data)-1)][0])
    for _ in range(len(data)):
        belong.append(-1)  #定义初始值-1表示尚未归属
    while True:
        change = False #标志簇中心是否发生变化
        for index, d in enumerate(data):
            dis = []  #距离列表
            for c in cluster:
                dis.append( distance(d[0], c) )
            if belong[index] != dis.index(min(dis)):
                change = True
                belong[index] = dis.index(min(dis))
        if change == False:
            break
        for index, c in enumerate(cluster):
            sum = 0
            num = 0
            for i, b in enumerate(belong):
                if index == b:
                    sum = sum + (data[i][0] * data[i][1])
                    num = num + data[i][1]
            if num == 0:
                c = 0
            else:
                c = sum/num
                cluster[index] = c
    for i in range(0,k):
        cluster[i] = round(cluster[i], 3)  # 设置聚类中心的精度到小数点后面三位
    return cluster,belong

def criterion(data, cluster, belong):
    #data::分数列表[[分数1， 人数1],...,[分数n， 人数n]]
    #cluster:聚类中心数组，长度为k
    #belong:分数隶属中心列表
    k = len(cluster)  # 获取聚类数
    WSS = 0  # WSS:Within cluster sum of squares,表示各个点到cluster中心距离的绝对值,代表误差
    for i in range(0,k):
        for index, b in enumerate(belong):
            if b == i:
                WSS += distance(cluster[i], data[index][0])
    return WSS;

#Part3 生成聚类信息并输入表格
#注:为方便测试,这里将输入表格信息目前放在主函数中,但在之后运行时可将其直接放入这个函数中
def clusterFromData(dataset):
    clusterList = []
    for data in dataset:
        if len(data)>5:  # 剔除不合规则的数据
            pointdata = data[1:]
            clusterCenter,belong = Clustering(pointdata, 5)
            clusterCenter.sort()
            clusterCenter.append(data[0])  # 加入标签
            for i in range(5,0,-1):
                clusterCenter[i] = clusterCenter[i-1]
            clusterCenter[0] = data[0]
            clusterList.append(clusterCenter)
    return clusterList


def get_dic(path='./db.sqlite3'):
    sql = 'select * from Provinces'
    rows = readFronSqllite(path, sql) 
    Dic = {}
    for row in rows:
        Dic[row[0]]=row[1]
    return Dic

def get_college_name(path='./db.sqlite3'):
    sql = 'select collegeID,collegeName from Colleges'
    rows = readFronSqllite(path, sql) 
    Dic = {}
    for row in rows:
        Dic[row[0]]=row[1]
    return Dic

In [2]:
#下面是主函数
dataset = get_dataset(path='./db.sqlite3')

In [8]:
center = clusterFromData(dataset)

In [9]:
cen=[]
newcen=[]
for i in range(len(center)):
    cen.append([])
    for j in range(1,6):
        cen[i].append(center[i][j])
for i in range(len(center)):
    cen[i].sort(reverse=True)

In [12]:
center

[[(1, 2017, 1), 531.0, 559.5, 587.475, 606.085, 630.483],
 [(1, 2017, 2), 494.659, 519.076, 546.571, 579.28, 622.482],
 [(1, 2018, 1), 554.018, 570.369, 588.07, 608.215, 633.879],
 [(1, 2018, 2), 476.807, 506.469, 539.436, 613.66, 649.874],
 [(2, 2017, 1), 356.69, 412.626, 467.281, 522.585, 585.314],
 [(2, 2017, 2), 399.2, 457.135, 511.088, 567.299, 629.507],
 [(2, 2018, 1), 612.747, 626.103, 640.17, 656.276, 674.226],
 [(2, 2018, 2), 605.794, 624.418, 642.521, 661.951, 682.241],
 [(2, 2019, 1), 558.606, 577.739, 596.526, 617.777, 640.37],
 [(2, 2019, 2), 514.834, 546.711, 580.288, 617.377, 660.584],
 [(3, 2018, 1), 595.855, 633.093, 675.06, 725.403, 797.131],
 [(3, 2018, 2), 557.998, 597.777, 641.581, 694.419, 770.222],
 [(4, 2017, 1), 282.484, 301.024, 318.897, 338.655, 364.704],
 [(4, 2017, 2), 286.025, 312.033, 332.77, 353.634, 378.588],
 [(4, 2018, 1), 288.112, 311.121, 331.253, 351.213, 375.032],
 [(4, 2018, 2), 295.757, 320.366, 340.363, 359.81, 382.25],
 [(4, 2019, 1), 283.757,

In [13]:
newcen1=[]
s=0
for j in range(1,35):
    for m in range(1,3):

        sum1=[]
        sum2=[]
        sum3=[]
        sum4=[]
        sum0=[]
        for i in range(len(center)):
            if int(list(center[i][0])[0])==j and int(list(center[i][0])[2])==m:
                sum0.append(cen[i][0])
                sum1.append(cen[i][1])
                sum2.append(cen[i][2])
                sum3.append(cen[i][3])
                sum4.append(cen[i][4])
        if(sum0!=[]):
            newcen1.append([])
            newcen1[s].append(j)
            newcen1[s].append(m)
            newcen1[s].append(sum(sum0)/len(sum0))
            newcen1[s].append(sum(sum1)/len(sum1))
            newcen1[s].append(sum(sum2)/len(sum2))
            newcen1[s].append(sum(sum3)/len(sum3))
            newcen1[s].append(sum(sum4)/len(sum4))
            s=s+1

In [14]:
newcen1

[[1, 1, 632.181, 607.1500000000001, 587.7725, 564.9345000000001, 542.509],
 [1, 2, 636.178, 596.47, 543.0035, 512.7725, 485.733],
 [2,
  1,
  633.3033333333333,
  598.8793333333333,
  567.9923333333332,
  538.8226666666666,
  509.3476666666666],
 [2,
  2,
  657.444,
  615.5423333333333,
  577.9656666666666,
  542.7546666666666,
  506.6093333333333],
 [3, 1, 797.131, 725.403, 675.06, 633.093, 595.855],
 [3, 2, 770.222, 694.419, 641.581, 597.777, 557.998],
 [4,
  1,
  370.4576666666667,
  345.3006666666667,
  325.3106666666667,
  306.08566666666667,
  284.78433333333334],
 [4,
  2,
  382.39366666666666,
  359.021,
  339.7186666666667,
  320.65566666666666,
  297.4483333333333],
 [6, 1, 539.8495, 491.3575, 453.91650000000004, 418.8245, 380.097],
 [6, 2, 577.533, 510.7915, 460.75849999999997, 416.4235, 369.9935],
 [10, 1, 592.9480000000001, 555.3495, 525.6735, 499.85749999999996, 475.565],
 [10, 2, 603.937, 551.3679999999999, 509.1055, 471.133, 434.31500000000005],
 [12, 1, 518.0755, 421.9

In [15]:
myinput=[1,1,555]

In [19]:
collegelist=[]
for d in dataset2:
    if d[0:2]==myinput[0:2]:
        collegelist.append(d)

In [36]:
collegelist

[[1, 1, '北京大学', 653.6666666666666, 88.73216666666656],
 [1, 1, '中国人民大学', 635.0704225352113, 70.1359225352112],
 [1, 1, '清华大学', 654.6666666666666, 89.73216666666656],
 [1, 1, '北京交通大学', 616.0, 51.06549999999993],
 [1, 1, '北京航空航天大学', 629.5, 64.56549999999993],
 [1, 1, '北京理工大学', 615.0, 50.06549999999993],
 [1, 1, '北京科技大学', 607.4666666666667, 42.532166666666626],
 [1, 1, '北方工业大学', 568.3333333333334, 3.3988333333333003],
 [1, 1, '北京化工大学', 596.0625, 31.12799999999993],
 [1, 1, '北京工商大学', 589.4285714285714, 24.494071428571374],
 [1, 1, '北京邮电大学', 605.3846153846154, 40.45011538461529],
 [1, 1, '北京林业大学', 590.6785714285714, 25.744071428571374],
 [1, 1, '北京师范大学', 621.6388888888889, 56.70438888888884],
 [1, 1, '首都师范大学', 592.3333333333334, 27.3988333333333],
 [1, 1, '北京外国语大学', 624.04, 59.10549999999989],
 [1, 1, '中国传媒大学', 615.7666666666667, 50.83216666666658],
 [1, 1, '华北电力大学', 599.4, 34.465499999999906],
 [1, 1, '太原理工大学', 578.3333333333334, 13.3988333333333],
 [1, 1, '内蒙古大学', 572.2926829268292, 7.358

In [27]:
for e in newcen1:
    if e[0]==myinput[0] and e[1]==myinput[1]:
        abslist=[]
        for i in range(2,7):
            abslist.append(abs(myinput[2]-e[i]))
        print(abslist)
        minnum=1000
        minj=0
        for j in range(5):
            if abslist[j]<minnum:
                minj=j
                minnum=abslist[j]
        target=e[minj+2]
        print("你属于第",minj+1,"类")

[77.18100000000004, 52.15000000000009, 32.772500000000036, 9.934500000000071, 12.490999999999985]
你属于第 4 类


In [28]:
target

564.9345000000001

In [33]:
for e in collegelist:
    e.append(abs(e[3]-target))

In [35]:
collegelist

[[1, 1, '北京大学', 653.6666666666666, 88.73216666666656],
 [1, 1, '中国人民大学', 635.0704225352113, 70.1359225352112],
 [1, 1, '清华大学', 654.6666666666666, 89.73216666666656],
 [1, 1, '北京交通大学', 616.0, 51.06549999999993],
 [1, 1, '北京航空航天大学', 629.5, 64.56549999999993],
 [1, 1, '北京理工大学', 615.0, 50.06549999999993],
 [1, 1, '北京科技大学', 607.4666666666667, 42.532166666666626],
 [1, 1, '北方工业大学', 568.3333333333334, 3.3988333333333003],
 [1, 1, '北京化工大学', 596.0625, 31.12799999999993],
 [1, 1, '北京工商大学', 589.4285714285714, 24.494071428571374],
 [1, 1, '北京邮电大学', 605.3846153846154, 40.45011538461529],
 [1, 1, '北京林业大学', 590.6785714285714, 25.744071428571374],
 [1, 1, '北京师范大学', 621.6388888888889, 56.70438888888884],
 [1, 1, '首都师范大学', 592.3333333333334, 27.3988333333333],
 [1, 1, '北京外国语大学', 624.04, 59.10549999999989],
 [1, 1, '中国传媒大学', 615.7666666666667, 50.83216666666658],
 [1, 1, '华北电力大学', 599.4, 34.465499999999906],
 [1, 1, '太原理工大学', 578.3333333333334, 13.3988333333333],
 [1, 1, '内蒙古大学', 572.2926829268292, 7.358

In [37]:
collegelist.sort(key=lambda x:x[4])
collegelist

[[1, 1, '上海师范大学', 565.5, 0.5654999999999291],
 [1, 1, '武汉科技大学', 564.3333333333334, 0.6011666666666997],
 [1, 1, '北京联合大学', 561.75, 3.184500000000071],
 [1, 1, '北方工业大学', 568.3333333333334, 3.3988333333333003],
 [1, 1, '延边大学', 569.0689655172414, 4.134465517241324],
 [1, 1, '长安大学', 560.8, 4.134500000000116],
 [1, 1, '新疆大学', 569.1071428571429, 4.172642857142819],
 [1, 1, '南通大学', 558.0, 6.934500000000071],
 [1, 1, '福建农林大学', 558.0, 6.934500000000071],
 [1, 1, '内蒙古大学', 572.2926829268292, 7.3581829268291585],
 [1, 1, '浙江工业大学', 572.6666666666666, 7.732166666666558],
 [1, 1, '浙江理工大学', 573.0, 8.065499999999929],
 [1, 1, '广西大学', 573.25, 8.315499999999929],
 [1, 1, '南京工业大学', 573.3333333333334, 8.3988333333333],
 [1, 1, '湖北大学', 574.3333333333334, 9.3988333333333],
 [1, 1, '燕山大学', 574.7619047619048, 9.827404761904745],
 [1, 1, '东北农业大学', 554.0, 10.934500000000071],
 [1, 1, '东北林业大学', 576.2222222222222, 11.2877222222221],
 [1, 1, '华侨大学', 576.4, 11.465499999999906],
 [1, 1, '青岛大学', 578.0, 13.0654999999999

In [39]:
print("为你推荐的大学是：")
for i in range(1,7,2):
    print(collegelist[i][2])
    

为你推荐的大学是：
武汉科技大学
北方工业大学
长安大学


In [5]:
rows = readFronSqllite('./db.sqlite3', 'select provinceID_id,categoryID_id,collegeID_id,avg(minScore) from Majors group by provinceID_id,collegeID_id,categoryID_id')
dataset2 = []
for row in rows:
    dataset2.append(list(row))
Dic = get_college_name(path='./db.sqlite3')
for d in dataset2:
    d[2] = Dic.get(d[2])
dataset2 

[[1, 1, '北京大学', 653.6666666666666],
 [1, 2, '北京大学', 681.0],
 [1, 1, '中国人民大学', 635.0704225352113],
 [1, 2, '中国人民大学', 654.8169014084507],
 [1, 1, '清华大学', 654.6666666666666],
 [1, 2, '清华大学', 675.8333333333334],
 [1, 1, '北京交通大学', 616.0],
 [1, 2, '北京交通大学', 616.25],
 [1, 1, '北京航空航天大学', 629.5],
 [1, 2, '北京航空航天大学', 661.9285714285714],
 [1, 1, '北京理工大学', 615.0],
 [1, 2, '北京理工大学', 630.9152542372881],
 [1, 1, '北京科技大学', 607.4666666666667],
 [1, 2, '北京科技大学', 609.2758620689655],
 [1, 1, '北方工业大学', 568.3333333333334],
 [1, 2, '北方工业大学', 568.6666666666666],
 [1, 1, '北京化工大学', 596.0625],
 [1, 2, '北京化工大学', 597.0175438596491],
 [1, 1, '北京工商大学', 589.4285714285714],
 [1, 2, '北京工商大学', 564.9142857142857],
 [1, 1, '北京邮电大学', 605.3846153846154],
 [1, 2, '北京邮电大学', 622.1166666666667],
 [1, 1, '北京林业大学', 590.6785714285714],
 [1, 2, '北京林业大学', 586.6010928961749],
 [1, 1, '北京师范大学', 621.6388888888889],
 [1, 2, '北京师范大学', 632.8153846153846],
 [1, 1, '首都师范大学', 592.3333333333334],
 [1, 2, '首都师范大学', 578.6666666666666],
 [1, 1, 